In [1]:
#!/usr/bin/env bash

<span style="color: red; font-weight: bold">Use the following command to launch a server for this notebook:</span>

```bash
git clone git@github.com:input-output-hk/marlowe-cardano.git
cd marlowe-cardano/marlowe-runtime/
nix run ../marlowe-cli
```

Then navigate to the `examples/` folder in Jupyter and open this notebook.

# Demonstrating the Marlowe Transaction Deposit Component of Marlowe Runtime

## Preliminaries

Record version numbers

In [2]:
marlowe-cli --version

marlowe-cli 0.0.9.0


In [3]:
cardano-cli --version

cardano-cli 1.35.3 - linux-x86_64 - ghc-8.10
git rev 0000000000000000000000000000000000000000


In [4]:
git rev-parse HEAD

a384fbf2e280f19290465cb0620a3116e04ac644


### Setup the faucet.

Set the location of keys.

In [5]:
TREASURY=treasury

Set the faucet.

In [6]:
FAUCET_SKEY=$TREASURY/payment.skey
FAUCET_ADDR=$(cat $TREASURY/payment.testnet.address)
echo "$FAUCET_ADDR"

addr_test1vq9prvx8ufwutkwxx9cmmuuajaqmjqwujqlp9d8pvg6gupczgtm9j


### Select network

In a separate terminal, set up a tunnel to the Marlowe Runtime development server:
```bash
rm /tmp/preview.socket
ssh -NT \
  -L/tmp/preview.socket:/data/networks/preview/node.socket \
  -L 3717:127.0.0.1:23717 \
  -L 3718:127.0.0.1:23718 \
  -L 3719:127.0.0.1:23719 \
  -L 3721:127.0.0.1:23721 \
  -L 3723:127.0.0.1:23723 \
  54.202.238.5                                              
```

In [7]:
export CARDANO_NODE_SOCKET_PATH=/tmp/preview.socket
export CARDANO_TESTNET_MAGIC=2
MAGIC=(--testnet-magic 2)
echo "${MAGIC[@]}"

--testnet-magic 2


### Time computations

In [8]:
SECOND=1000
MINUTE=$((60 * SECOND))

In [9]:
NOW="$(($(date -u +%s) * SECOND))"
echo "$NOW"

1666111908000


### Check that the reference script has been published

Check that the Marlowe semantics validator was published.

In [10]:
marlowe-cli transaction find-published


Searching for reference script at address: addr_test1vrw0tuh8l95thdqr65dmpcfqnmcw0en7v7vhgegck7gzqgswa07sw

Expected reference script hash: "6a9391d6aa51af28dd876ebb5565b69d1e83e5ac7861506bd29b56b0"

Searching for reference script at address: addr_test1vpa36uuyf95kxpcleldsncedlhjru6vdmh2vnpkdrsz4u6cll9zas

Expected reference script hash: "49076eab20243dc9462511fb98a9cfb719f86e9692288139b7c91df3"
{
    "marlowe": {
        "hash": "6a9391d6aa51af28dd876ebb5565b69d1e83e5ac7861506bd29b56b0",
        "txIn": "087f21f109a997193421a81886ea8c6397d336d19e696457b9c5c7aefdc31873#1"
    },
    "payout": {
        "hash": "49076eab20243dc9462511fb98a9cfb719f86e9692288139b7c91df3",
        "txIn": "087f21f109a997193421a81886ea8c6397d336d19e696457b9c5c7aefdc31873#2"
    }
}


### Participants

#### The Party

In [11]:
PARTY_SKEY="$TREASURY/john-fletcher.skey"
PARTY_VKEY="$TREASURY/john-fletcher.vkey"

Create the first party's keys, if necessary.

In [12]:
if [[ ! -e "$PARTY_SKEY" ]]
then
  cardano-cli address key-gen --signing-key-file "$PARTY_SKEY" --verification-key-file "$PARTY_VKEY"
fi
PARTY_ADDR=$(cardano-cli address build "${MAGIC[@]}" --payment-verification-key-file "$PARTY_VKEY")
echo "$PARTY_ADDR"

addr_test1vqwt2xlr4d8yk4qws675exlqy6pdhq2s76wrehkjggkvr0cerfe8r


Fund the address.

In [13]:
marlowe-cli util fund-address \
  --out-file /dev/null \
  --submit 600 \
  --lovelace 250000000 \
  --source-wallet-credentials "$FAUCET_ADDR:$FAUCET_SKEY" \
  "$PARTY_ADDR"

TxId "af1cc39a41ba521ad66b925dfeba9f292d9de284d8ba4951b03fa0868df66f6b"


#### The Counterparty

In [14]:
COUNTERPARTY_SKEY="$TREASURY/thomas-kyd.skey"
COUNTERPARTY_VKEY="$TREASURY/thomas-kyd.vkey"

Create the second party's keys, if necessary.

In [15]:
if [[ ! -e "$COUNTERPARTY_SKEY" ]]
then
  cardano-cli address key-gen --signing-key-file "$COUNTERPARTY_SKEY" --verification-key-file "$COUNTERPARTY_VKEY"
fi
COUNTERPARTY_ADDR=$(cardano-cli address build "${MAGIC[@]}" --payment-verification-key-file "$COUNTERPARTY_VKEY")
echo "$COUNTERPARTY_ADDR"

addr_test1vr7n0zzth5zycuh972w7rdmh48qur4f3wu6ntn2m2h30dlcvltuy5


Fund the address.

In [16]:
marlowe-cli util fund-address \
  --out-file /dev/null \
  --submit 600  \
  --lovelace 250000000 \
  --source-wallet-credentials "$FAUCET_ADDR:$FAUCET_SKEY" \
  "$COUNTERPARTY_ADDR"

TxId "c8c5d98e6509d70d99b28b4ac9d09e93f8861749186ef769457a77c6d71204f7"


## The Contract

We set the parameters for the ACTUS PAM contract.

In [17]:
MINIMUM_ADA=3000000

#FIXED_POINT=1000000
PRINCIPAL=100
INTEREST_RATE=0.02

STATUS_DATE=$(date -d "$(date -u -R -d @$((NOW/1000)))" +"%Y-%m-%dT00:00:00")
INITIAL_EXCHANGE_DATE=$(date -d "$(date -u -R -d @$((NOW/1000))) + 1 year" +"%Y-01-01T00:00:00")
MATURITY_DATE=$(date -d "$(date -u -R -d @$((NOW/1000))) + 2 year" +"%Y-01-01T00:00:00")

In [18]:
yaml2json << EOI > deposit.actus
scheduleConfig:
  businessDayConvention: "NULL"
  endOfMonthConvention: "EOM"
  calendar: "NC"
maturityDate: "$MATURITY_DATE"
contractId: "0"
enableSettlement: false
initialExchangeDate: "$INITIAL_EXCHANGE_DATE"
contractRole: "RPA"
penaltyType: "O"
cycleAnchorDateOfInterestPayment: "$INITIAL_EXCHANGE_DATE"
contractType: "PAM"
notionalPrincipal: $PRINCIPAL
contractPerformance: "PF"
collateralAmount: 0
dayCountConvention: "30E360"
accruedInterest: 0
statusDate: "$STATUS_DATE"
cycleOfInterestPayment: "P1YL1"
prepaymentEffect: "N"
nominalInterestRate: $INTEREST_RATE
interestCalculationBase: "NT"
EOI
cat deposit.actus

{"accruedInterest":0,"collateralAmount":0,"contractId":"0","contractPerformance":"PF","contractRole":"RPA","contractType":"PAM","cycleAnchorDateOfInterestPayment":"2023-01-01T00:00:00","cycleOfInterestPayment":"P1YL1","dayCountConvention":"30E360","enableSettlement":false,"initialExchangeDate":"2023-01-01T00:00:00","interestCalculationBase":"NT","maturityDate":"2024-01-01T00:00:00","nominalInterestRate":0.02,"notionalPrincipal":100,"penaltyType":"O","prepaymentEffect":"N","scheduleConfig":{"businessDayConvention":"NULL","calendar":"NC","endOfMonthConvention":"EOM"},"statusDate":"2022-10-18T00:00:00"}


Create the contract.

In [19]:
marlowe-cli template actus \
  --minimum-ada "$MINIMUM_ADA" \
  --party "$PARTY_ADDR" \
  --counter-party "$COUNTERPARTY_ADDR" \
  --actus-terms-file  deposit.actus \
  --out-contract-file deposit-1.contract \
  --out-state-file /dev/null

Since we are testing, we don't really want to wait months or years for timeouts, so we edit the contract file to change the maturity date to 10 minutes from now.

In [20]:
sed -i -e "s/$(($(date -d "$MATURITY_DATE" -u +%s) * SECOND))/$((NOW + 10 * MINUTE))/" deposit-1.contract

View the contract.

In [21]:
json2yaml deposit-1.contract

timeout: 1672531200000
timeout_continuation: close
when:
- case:
    deposits:
      negate:
        negate: 100000000
    into_account:
      address: addr_test1vqwt2xlr4d8yk4qws675exlqy6pdhq2s76wrehkjggkvr0cerfe8r
    of_token:
      currency_symbol: ''
      token_name: ''
    party:
      address: addr_test1vqwt2xlr4d8yk4qws675exlqy6pdhq2s76wrehkjggkvr0cerfe8r
  then:
    from_account:
      address: addr_test1vqwt2xlr4d8yk4qws675exlqy6pdhq2s76wrehkjggkvr0cerfe8r
    pay: 100000000
    then:
      timeout: 1666112508000
      timeout_continuation: close
      when:
      - case:
          deposits: 2000000
          into_account:
            address: addr_test1vr7n0zzth5zycuh972w7rdmh48qur4f3wu6ntn2m2h30dlcvltuy5
          of_token:
            currency_symbol: ''
            token_name: ''
          party:
            address: addr_test1vr7n0zzth5zycuh972w7rdmh48qur4f3wu6ntn2m2h30dlcvltuy5
        then:
          from_account:
            address: addr_test1vr7n0zzth5zycuh972w7rdm

## Run the Contract

### Transaction 1. Create the ontract

See what UTxOs the transaction-creation will have available to select from.

In [22]:
cardano-cli query utxo "${MAGIC[@]}" --address "$PARTY_ADDR"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
af1cc39a41ba521ad66b925dfeba9f292d9de284d8ba4951b03fa0868df66f6b     1        250000000 lovelace + TxOutDatumNone


Build the transaction.

In [23]:
CONTRACT_ID=$(
marlowe create \
  --core-file deposit-1.contract \
  --min-utxo "$MINIMUM_ADA" \
  --change-address "$PARTY_ADDR" \
  --address "$PARTY_ADDR" \
  --manual-sign deposit-1.txbody \
| sed -e 's/^.*"\([^\\]*\)\\.*$/\1/' \
)
echo "CONTRACT_ID = $CONTRACT_ID"

CONTRACT_ID = 90d06a8977ec4d5460ad50b95472b914ebb64d143e5466018ff2db319b78599c#1


Sign the transaction.

In [24]:
cardano-cli transaction sign \
  --tx-body-file deposit-1.txbody \
  --out-file     deposit-1.tx \
  --signing-key-file "$PARTY_SKEY"

Submit the transaction using Marlowe Runtime.

In [25]:
marlowe submit deposit-1.tx

"{\"blockHeaderHash\":\"c2f228091987027c6f33c3edc361f2e67b5953d1d0b00abd3ff495984f182107\",\"blockNo\":282058,\"slotNo\":6108743}"


Watch the contract.

In [26]:
marlowe add "$CONTRACT_ID"

90d06a8977ec4d5460ad50b95472b914ebb64d143e5466018ff2db319b78599c#1


In [27]:
marlowe log --show-contract "$CONTRACT_ID"

transaction 90d06a8977ec4d5460ad50b95472b914ebb64d143e5466018ff2db319b78599c (creation)
ContractId:      90d06a8977ec4d5460ad50b95472b914ebb64d143e5466018ff2db319b78599c#1
SlotNo:          6108743
BlockNo:         282058
BlockId:         c2f228091987027c6f33c3edc361f2e67b5953d1d0b00abd3ff495984f182107
ScriptAddress:   addr_test1wp4f8ywk4fg672xasahtk4t9k6w3aql943uxz5rt62d4dvqu3c6jv
Marlowe Version: 1

    When [
      (Case
         (Deposit Address "addr_test1vqwt2xlr4d8yk4qws675exlqy6pdhq2s76wrehkjggkvr0cerfe8r" Address "addr_test1vqwt2xlr4d8yk4qws675exlqy6pdhq2s76wrehkjggkvr0cerfe8r"
            (Token "" "")
            (NegValue
               (NegValue
                  (Constant 100000000))))
         (Pay Address "addr_test1vqwt2xlr4d8yk4qws675exlqy6pdhq2s76wrehkjggkvr0cerfe8r"
            (Party Address "addr_test1vr7n0zzth5zycuh972w7rdmh48qur4f3wu6ntn2m2h30dlcvltuy5")
            (Token "" "")
            (Constant 100000000)
            (When [
               (Case
          

View the contract's UTxO.

In [28]:
CONTRACT_ADDRESS=$(marlowe-cli contract address)
echo "$CONTRACT_ADDRESS"

addr_test1wp4f8ywk4fg672xasahtk4t9k6w3aql943uxz5rt62d4dvqu3c6jv


In [29]:
cardano-cli query utxo "${MAGIC[@]}" --address "$CONTRACT_ADDRESS" | sed -n -e "1,2p;/${CONTRACT_ID//#*/}/p"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
90d06a8977ec4d5460ad50b95472b914ebb64d143e5466018ff2db319b78599c     1        3000000 lovelace + TxOutDatumHash ScriptDataInBabbageEra "57f9b63c79cfe84ff2df9bcab650fcf4dc0c58f96ec020512d41c4f2a7c73bdd"


### Transaction 2. Party deposits loan amount

The party deposits the loan amount.

In [30]:
marlowe deposit \
  --contract "$CONTRACT_ID" \
  --from-party "$PARTY_ADDR" \
  --to-party "$PARTY_ADDR" \
  --lovelace "$((PRINCIPAL*FIXED_POINT))" \
  --validity-lower-bound "$((NOW-5*MINUTE))" \
  --validity-upper-bound "$((NOW+9*MINUTE))" \
  --change-address "$PARTY_ADDR" \
  --address "$PARTY_ADDR" \
  --manual-sign deposit-2.txbody

Tx client client failure
marlowe: DeserializeError {message = "not enough bytes", offset = 0, unconsumedInput = ""}


: 1

<span style="color:red;font-weight:bold">It looks like we have a deserialization error.</span>

Sign the transaction.

In [ ]:
cardano-cli transaction sign \
  --tx-body-file deposit-2.txbody \
  --out-file     deposit-2.tx \
  --signing-key-file "$PARTY_SKEY"

Submit the transaction using Marlowe Runtime.

In [ ]:
marlowe submit deposit-2.tx

In [ ]:
marlowe log --show-contract "$CONTRACT_ID"

View the contract's UTxO.

In [ ]:
cardano-cli query utxo "${MAGIC[@]}" --address "$CONTRACT_ADDRESS" | sed -n -e "1,2p;/${CONTRACT_ID//#*/}/p"

### Transaction 3. Counterparty repays the loan's interest

The counterparty repays the interest.

In [ ]:
marlowe deposit \
  --contract "$CONTRACT_ID" \
  --from-party "$COUNTERPARTY_ADDR" \
  --to-party "$COUNTERPARTY_ADDR" \
  --lovelace "$((INTEREST*FIXED_POINT))" \
  --validity-lower-bound "$((NOW-5*MINUTE))" \
  --validity-upper-bound "$((NOW+9*MINUTE))" \
  --change-address "$COUNTERPARTY_ADDR" \
  --address "$COUNTERPARTY_ADDR" \
  --manual-sign deposit-3.txbody

Sign the transaction.

In [ ]:
cardano-cli transaction sign \
  --tx-body-file deposit-3.txbody \
  --out-file     deposit-3.tx \
  --signing-key-file "$COUNTERPARTY_SKEY"

Submit the transaction using Marlowe Runtime.

In [ ]:
marlowe submit deposit-3.tx

In [ ]:
marlowe log --show-contract "$CONTRACT_ID"

View the contract's UTxO.

In [ ]:
cardano-cli query utxo "${MAGIC[@]}" --address "$CONTRACT_ADDRESS" | sed -n -e "1,2p;/${CONTRACT_ID//#*/}/p"

### Transaction 4. Counterparty repays the loan's principal

The counterparty repays the principal.

In [ ]:
marlowe deposit \
  --contract "$CONTRACT_ID" \
  --from-party "$COUNTERPARTY_ADDR" \
  --to-party "$COUNTERPARTY_ADDR" \
  --lovelace "$((PRINCIPAL*FIXED_POINT))" \
  --validity-lower-bound "$((NOW-5*MINUTE))" \
  --validity-upper-bound "$((NOW+9*MINUTE))" \
  --change-address "$COUNTERPARTY_ADDR" \
  --address "$COUNTERPARTY_ADDR" \
  --manual-sign deposit-4.txbody

Sign the transaction.

In [ ]:
cardano-cli transaction sign \
  --tx-body-file deposit-4.txbody \
  --out-file     deposit-4.tx \
  --signing-key-file "$COUNTERPARTY_SKEY"

Submit the transaction using Marlowe Runtime.

In [ ]:
marlowe submit deposit-4.tx

In [ ]:
marlowe log --show-contract "$CONTRACT_ID"

View the contract's UTxO.

In [ ]:
cardano-cli query utxo "${MAGIC[@]}" --address "$CONTRACT_ADDRESS" | sed -n -e "1,2p;/${CONTRACT_ID//#*/}/p"

## Cleanup

Remove the contract from history tracking.

In [31]:
marlowe rm "$CONTRACT_ID"

90d06a8977ec4d5460ad50b95472b914ebb64d143e5466018ff2db319b78599c#1


In [32]:
marlowe ls

Consolidate the UTxOs at the addresses.

In [33]:
marlowe-cli util clean \
  --change-address "$PARTY_ADDR" \
  --required-signer "$PARTY_SKEY" \
  --out-file /dev/null \
  --submit 600

TxId "d50c427b5844f9debe61f6366d89ef8ef7133bbe52852d1d8aeace4d020f48c9"


In [34]:
marlowe-cli util clean \
  --change-address "$COUNTERPARTY_ADDR" \
  --required-signer "$COUNTERPARTY_SKEY" \
  --out-file /dev/null \
  --submit 600

TxId "fd8a40b29ad3221b652df924c5b19f929bd1ddf79f0859c22420df6ac5868a7e"


Send the funds back to the faucet.

In [35]:
marlowe-cli transaction simple \
  --tx-in "$(marlowe-cli util select --lovelace-only 1 "$PARTY_ADDR" | sed -n -e 's/^TxIn "\(.*\)" (TxIx \(.*\))$/\1#\2/;1p')" \
  --tx-in "$(marlowe-cli util select --lovelace-only 1 "$COUNTERPARTY_ADDR" | sed -n -e 's/^TxIn "\(.*\)" (TxIx \(.*\))$/\1#\2/;1p')" \
  --required-signer "$PARTY_SKEY" \
  --required-signer "$COUNTERPARTY_SKEY" \
  --change-address "$FAUCET_ADDR" \
  --out-file /dev/null \
  --submit 600

TxId "81209f764efe87123af74a29b20d1ddd11f2d2ae03e971bd7f5d1e541144afd1"


See that the funds have been returned to the faucet.

In [36]:
cardano-cli query utxo "${MAGIC[@]}" --address "$PARTY_ADDR" --address "$COUNTERPARTY_ADDR"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
